# Background


***Context-aware multi-modal transportation recommendation*** has a goal of recommending a travel plan which considers various unimodal transportation modes, such as walking, cycling, driving, public transit, and how to connect among these modes under various contexts. The successful development of multi-modal transportation recommendations can have a number of advantages, including but not limited to reducing transport times, balancing traffic flows, reducing traffic congestion, and ultimately, promoting the development of intelligent transportation systems.

Despite the popularity and frequent usage of transportation recommendation on navigation Apps (e.g., Baidu Maps and Google Maps), ***existing transportation recommendation solutions only consider routes in one transportation mode.*** Intuitively, in the context-aware multi-modal transportation recommendation problem, the transport mode preferences vary over ***different users and spatiotemporal contexts***. For example, metros are more cost-effective than taxis for most urban commuters; economically disadvantaged people may prefer cycling and walking to others for local travel, if the transport options are inadequate. Imagine another scenario that the distance of the OD pair is relatively large, and the trip purpose is in no emergency. In this case, a ***cost-effective transportation recommendation*** that including multiple transport modes, e.g., taxi-bus, maybe more attractive.

# Tasks

**Task 1: Recommend the most appropriate transport mode** (Phase 1 Deadline is ***2019-05-30***)
This first challenge will address the context-aware multi-modal transportation recommendation problem. Given a user u an ***OD pair and the situational context***, it is requested to recommend the most proper transport mode m ∈  M for user u to travel between the OD pair od. 

**Task 2: Open research/application challenge**
The published dataset has great potential to be applied in various research tasks and applications. The second task will be an open challenge based on the published dataset. A research/application proposal is required to explain:

1) The proposed research/application topic, including the aim of the research/application, the context and the justification.

2) The general methodology and expected resources and conditions to complete the research/application. A demo/video to help explain the final output is encouraged to submit.

3) The importance of your research/application topic.

# Task 1 Data

Task 1 requires participants to use ***historical user behavior data*** collected from Baidu Maps and a set of ***user attributes data*** to recommend the ***appropriate transport mode***. User behavior data capture user interactions with navigation applications. According to a user interaction loop, the user behavior data can be further categorized into query records, display records and click records. Each record is associated with a session ID and a timestamp. The session ID nexus the query, display plans and the user clicks correspondingly.

1) **Query record**. A query record represents one route search from a user on Baidu Maps. Each query record consists of a ***session ID***, a ***profile ID***, a ***timestamp***, coordinates of the ***original point***, coordinates of the ***destination point***. For example, [387056, 234590, "2018-11-01 15:15:36", (116.30,40.05), (116.35,39.99)] means a user makes a query on a trip from (116.30,40.05) to (116.35,39.99) in the afternoon of Nov. 1st, 2018. All coordinates are WGS84.

2) **Display record**. A display record is the feasible routes generated by Baidu Maps shown to the user. Each display record consists of a ***session ID***, a ***timestamp*** and a list of ***route plans***. Each display plan consists of the ***transport mode, the estimated route distance in meters, the estimated time of arrival (ETA) in seconds, the estimated price in RMB cent and implicitly the display rank in the display list***. To avoid confusion, there is at most one plan of a specific transport mode in the display list. There are 11 transport modes in total. A transport mode could be unimodal (e.g., drive, bus, cycle) or multi-modal (e.g., taxi-bus, cycle-bus), we encode these transport modes into numerical labels range from 1 to 11. For example, [387056, "2018-11-01 15:15:40", [{“mode”:1, “distance”: 3220, “ETA”: 2134, “price”: 12}, {“mode”:3, “distance”: 3520, “ETA”: 2841, “price”: 2}]] is a display record of two transport mode plans.

3) **Click record**. A click record indicates the user feedback of different recommendations, i.e., a user may click on specific routes displayed to him/her for details. In each record, click data contains a session ID, a timestamp, and the clicked transport mode in the display list. We only preserve the first click for each query.

4) **User attributes**. User profile attributes reflect individual preference on transport modes. The user of each session is associated with a set of user attributes via a profile ID. Each profile record consists of a ***profile ID***, a set of ***one hot encoded user profile dimensions***. Note that for the privacy issue, we don’t directly provide physically individual user IDs. Instead, each user is represented as a set of user attributes and then users with ***same attributes are merged with the same user profile ID***. For example, with gender and age attribute considered, two males of age 35 are identified as the same user in the dataset.

# Task 1 Result Format

**Task 1**
Each team only has two chances to submit every day to avoid improving results by simply enumerating all possible results of the test data. For each session in the test set, you should predict a transport mode that the user prefers to click. In the dataset, some queries have no clicks, we assign a none-click class (encoded as 0) to such queries and participants should also predict the none-click class as well.

The file should be a csv file with a header, the session id in the first column and the prediction in the second column, have the following format (submission_example.csv): 

“sid”, “recommend_mode”

“387056”, “0”

“902489”, “3”

“156976”, “11”

“183026”, “9”

“729561”, “1”


# Train Data

In [59]:
import pandas as pd
import numpy as np
import datetime
from geopy.distance import geodesic

In [60]:
train_data = pd.read_csv('train_data.csv')

In [61]:
train_data.head()

,Unnamed: 0,sid,pid,req_time,o,d,plan_time,plans,p0,p1,...,p58,p59,p60,p61,p62,p63,p64,p65,click_time,click_mode
0,0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96",2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-02 17:54:32,9.0
1,1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79",2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2018-11-16 10:53:32,1.0
2,2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00",2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-06 10:34:23,9.0
3,3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90",2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-11-23 14:54:18,1.0
4,4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86",2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2018-10-30 11:49:04,7.0


In [62]:
train_data['lng_o']=train_data['o'].apply(lambda x: float(x.split(',')[0]))
train_data['lat_o']=train_data['o'].apply(lambda x: float(x.split(',')[1]))
train_data['lng_d']=train_data['d'].apply(lambda x: float(x.split(',')[0]))
train_data['lat_d']=train_data['d'].apply(lambda x: float(x.split(',')[1]))

In [63]:
train_data.drop(['Unnamed: 0','o','d'],axis=1,inplace=True)

In [64]:
train_data['line_distance']=train_data.apply(lambda row: int(geodesic((row['lat_o'], row['lng_o']),(row['lat_d'],row['lng_d'])).m),axis=1)

In [65]:
train_data['req_time']=pd.to_datetime(train_data.req_time)
train_data['plan_time']=pd.to_datetime(train_data.plan_time)
train_data['click_time']=pd.to_datetime(train_data.click_time)

In [71]:
train_data.click_mode.fillna(0,inplace=True)

In [74]:
train_data['click_mode']=train_data.click_mode.astype(int)

In [76]:
train_data.head()

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p63,p64,p65,click_time,click_mode,lng_o,lat_o,lng_d,lat_d,line_distance
0,3000821,NaN,2018-11-02 17:54:30,2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-02 17:54:32,9,116.29,39.97,116.32,39.96,2793
1,3085857,210736.0,2018-11-16 10:53:10,2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,2018-11-16 10:53:32,1,116.39,39.84,116.33,39.79,7563
2,2944522,NaN,2018-10-06 10:33:58,2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-06 10:34:23,9,116.31,39.93,116.27,40.00,8490
3,559931,202427.0,2018-11-23 14:54:11,2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2018-11-23 14:54:18,1,116.27,39.88,116.39,39.90,10501
4,2819352,172251.0,2018-10-30 11:48:41,2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2018-10-30 11:49:04,7,116.34,39.96,116.37,39.86,11395


In [98]:
train_with_profile=train_data[train_data.pid.notnull()]
train_with_profile.reset_index(drop=True,inplace=True)

In [99]:
for i in range(66):
    train_with_profile['p{}'.format(i)]=train_with_profile['p{}'.format(i)].astype(int)

/Users/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [101]:
train_with_profile['pid']=train_with_profile.pid.astype(int)

/Users/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
train_with_profile.head()

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p63,p64,p65,click_time,click_mode,lng_o,lat_o,lng_d,lat_d,line_distance
0,3085857,210736,2018-11-16 10:53:10,2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0,0,0,0,1,...,1,0,0,2018-11-16 10:53:32,1,116.39,39.84,116.33,39.79,7563
1,559931,202427,2018-11-23 14:54:11,2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1,0,0,0,0,...,0,0,0,2018-11-23 14:54:18,1,116.27,39.88,116.39,39.90,10501
2,2819352,172251,2018-10-30 11:48:41,2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0,0,0,1,0,...,0,0,0,2018-10-30 11:49:04,7,116.34,39.96,116.37,39.86,11395
3,2754575,135606,2018-10-18 07:37:32,2018-10-18 07:37:32,"[{""distance"": 8217, ""price"": """", ""eta"": 1743, ...",0,0,0,0,1,...,0,0,0,2018-10-18 07:38:25,1,116.54,39.73,116.48,39.76,6127
4,2224795,189023,2018-11-20 14:34:03,2018-11-20 14:34:03,"[{""distance"": 99097, ""price"": """", ""eta"": 6596,...",1,0,0,0,0,...,0,0,0,2018-11-20 14:34:48,1,116.34,39.73,116.79,40.35,78829


In [103]:
train_without_profile=train_data[train_data.pid.isnull()]
train_without_profile.reset_index(drop=True,inplace=True)

In [104]:
train_without_profile

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p63,p64,p65,click_time,click_mode,lng_o,lat_o,lng_d,lat_d,line_distance
0,3000821,NaN,2018-11-02 17:54:30,2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-02 17:54:32,9,116.29,39.97,116.32,39.96,2793
1,2944522,NaN,2018-10-06 10:33:58,2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-06 10:34:23,9,116.31,39.93,116.27,40.00,8490
2,1677052,NaN,2018-10-23 01:02:45,2018-10-23 01:02:45,"[{""distance"": 21605, ""price"": 500, ""eta"": 5549...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-23 01:02:55,1,116.43,39.95,116.37,39.86,11233
3,986743,NaN,2018-11-12 13:52:31,2018-11-12 13:52:31,"[{""distance"": 30603, ""price"": 600, ""eta"": 4751...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-12 13:52:51,9,116.32,39.98,116.58,39.91,23540
4,184189,NaN,2018-10-20 12:52:07,2018-10-20 12:52:07,"[{""distance"": 4411, ""price"": 400, ""eta"": 2373,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-20 12:52:10,1,116.31,40.03,116.33,40.01,2801
5,3292454,NaN,2018-10-16 09:06:07,2018-10-16 09:06:07,"[{""distance"": 3437, ""price"": 1400, ""eta"": 858,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-16 09:06:40,1,116.30,39.94,116.27,39.96,3391
6,627704,NaN,2018-10-28 09:46:40,2018-10-28 09:46:40,"[{""distance"": 26609, ""price"": 600, ""eta"": 4057...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-28 09:46:53,2,116.29,39.86,116.45,39.93,15737
7,1548874,NaN,2018-11-28 08:34:30,2018-11-28 08:34:30,"[{""distance"": 5556, ""price"": 300, ""eta"": 2185,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-28 08:34:37,2,116.43,39.90,116.43,39.94,4441
8,1300125,NaN,2018-11-06 16:48:51,2018-11-06 16:48:51,"[{""distance"": 4366, ""price"": 200, ""eta"": 2056,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-06 16:48:57,1,116.51,39.92,116.55,39.91,3595
9,1806930,NaN,2018-11-29 11:08:52,2018-11-29 11:08:52,"[{""distance"": 22129, ""price"": 500, ""eta"": 4599...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-29 11:09:04,2,116.31,40.03,116.46,39.96,14983


# Test Data

In [77]:
qurey_test_df=pd.read_csv('/Users/martin/Downloads/data_set_phase1/test_queries.csv', delimiter=',')
display_test_df=pd.read_csv('/Users/martin/Downloads/data_set_phase1/test_plans.csv',delimiter=',')
profile_df=pd.read_csv('/Users/martin/Downloads/data_set_phase1/profiles.csv',delimiter=',')

In [78]:
result_mid=pd.merge(qurey_test_df, display_test_df, how='left', on=['sid'])
test_data=pd.merge(result_mid, profile_df, how='left', on=['pid'])

In [80]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94358 entries, 0 to 94357
Data columns (total 73 columns):
sid          94358 non-null int64
pid          62911 non-null float64
req_time     94358 non-null object
o            94358 non-null object
d            94358 non-null object
plan_time    92571 non-null object
plans        92571 non-null object
p0           62911 non-null float64
p1           62911 non-null float64
p2           62911 non-null float64
p3           62911 non-null float64
p4           62911 non-null float64
p5           62911 non-null float64
p6           62911 non-null float64
p7           62911 non-null float64
p8           62911 non-null float64
p9           62911 non-null float64
p10          62911 non-null float64
p11          62911 non-null float64
p12          62911 non-null float64
p13          62911 non-null float64
p14          62911 non-null float64
p15          62911 non-null float64
p16          62911 non-null float64
p17          62911 non-null float6

In [109]:
test_data.head()

,sid,pid,req_time,o,d,plan_time,plans,p0,p1,p2,...,p60,p61,p62,p63,p64,p65,lng_o,lat_o,lng_d,lat_d
0,1126541,178395.0,2018-12-05 14:51:43,"116.41,39.92","116.46,39.95",2018-12-05 14:51:43,"[{""distance"": 13610, ""price"": 400, ""eta"": 2388...",0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,116.41,39.92,116.46,39.95
1,453685,179178.0,2018-12-01 12:50:33,"116.39,39.90","116.41,39.91",2018-12-01 12:50:33,"[{""distance"": 2858, ""price"": 200, ""eta"": 1798,...",1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,116.39,39.90,116.41,39.91
2,2066073,184748.0,2018-12-07 10:17:35,"116.37,39.71","116.31,39.89",2018-12-07 10:17:35,"[{""distance"": 33453, ""price"": 800, ""eta"": 5624...",1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,116.37,39.71,116.31,39.89
3,452746,162795.0,2018-12-01 07:52:44,"116.33,39.97","116.31,39.98",2018-12-01 07:52:44,"[{""distance"": 4007, ""price"": 1500, ""eta"": 989,...",0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,116.33,39.97,116.31,39.98
4,1431672,116384.0,2018-12-06 10:36:30,"116.51,39.68","116.51,39.68",2018-12-06 10:36:30,"[{""distance"": 692, ""price"": """", ""eta"": 591, ""t...",0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,116.51,39.68,116.51,39.68


In [108]:
test_data['lng_o']=test_data['o'].apply(lambda x: float(x.split(',')[0]))
test_data['lat_o']=test_data['o'].apply(lambda x: float(x.split(',')[1]))
test_data['lng_d']=test_data['d'].apply(lambda x: float(x.split(',')[0]))
test_data['lat_d']=test_data['d'].apply(lambda x: float(x.split(',')[1]))

In [111]:
test_data.drop(['o','d'],axis=1,inplace=True)

In [112]:
test_data['line_distance']=test_data.apply(lambda row: int(geodesic((row['lat_o'], row['lng_o']),(row['lat_d'],row['lng_d'])).m),axis=1)

In [114]:
test_data['req_time']=pd.to_datetime(test_data.req_time)
test_data['plan_time']=pd.to_datetime(test_data.plan_time)

In [115]:
test_data.head()

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p61,p62,p63,p64,p65,lng_o,lat_o,lng_d,lat_d,line_distance
0,1126541,178395.0,2018-12-05 14:51:43,2018-12-05 14:51:43,"[{""distance"": 13610, ""price"": 400, ""eta"": 2388...",0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,116.41,39.92,116.46,39.95,5418
1,453685,179178.0,2018-12-01 12:50:33,2018-12-01 12:50:33,"[{""distance"": 2858, ""price"": 200, ""eta"": 1798,...",1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,116.39,39.90,116.41,39.91,2039
2,2066073,184748.0,2018-12-07 10:17:35,2018-12-07 10:17:35,"[{""distance"": 33453, ""price"": 800, ""eta"": 5624...",1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,116.37,39.71,116.31,39.89,20635
3,452746,162795.0,2018-12-01 07:52:44,2018-12-01 07:52:44,"[{""distance"": 4007, ""price"": 1500, ""eta"": 989,...",0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,116.33,39.97,116.31,39.98,2037
4,1431672,116384.0,2018-12-06 10:36:30,2018-12-06 10:36:30,"[{""distance"": 692, ""price"": """", ""eta"": 591, ""t...",0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,116.51,39.68,116.51,39.68,0


In [116]:
test_with_profile=test_data[test_data.pid.notnull()]
test_with_profile.reset_index(drop=True,inplace=True)

In [118]:
for i in range(66):
    test_with_profile['p{}'.format(i)]=test_with_profile['p{}'.format(i)].astype(int)

/Users/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [119]:
test_with_profile['pid']=test_with_profile.pid.astype(int)

/Users/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [120]:
train_with_profile.head()

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p63,p64,p65,click_time,click_mode,lng_o,lat_o,lng_d,lat_d,line_distance
0,3085857,210736,2018-11-16 10:53:10,2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",0,0,0,0,1,...,1,0,0,2018-11-16 10:53:32,1,116.39,39.84,116.33,39.79,7563
1,559931,202427,2018-11-23 14:54:11,2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",1,0,0,0,0,...,0,0,0,2018-11-23 14:54:18,1,116.27,39.88,116.39,39.90,10501
2,2819352,172251,2018-10-30 11:48:41,2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",0,0,0,1,0,...,0,0,0,2018-10-30 11:49:04,7,116.34,39.96,116.37,39.86,11395
3,2754575,135606,2018-10-18 07:37:32,2018-10-18 07:37:32,"[{""distance"": 8217, ""price"": """", ""eta"": 1743, ...",0,0,0,0,1,...,0,0,0,2018-10-18 07:38:25,1,116.54,39.73,116.48,39.76,6127
4,2224795,189023,2018-11-20 14:34:03,2018-11-20 14:34:03,"[{""distance"": 99097, ""price"": """", ""eta"": 6596,...",1,0,0,0,0,...,0,0,0,2018-11-20 14:34:48,1,116.34,39.73,116.79,40.35,78829


In [122]:
test_without_profile=test_data[test_data.pid.isnull()]
test_without_profile.reset_index(drop=True,inplace=True)

In [123]:
train_without_profile

,sid,pid,req_time,plan_time,plans,p0,p1,p2,p3,p4,...,p63,p64,p65,click_time,click_mode,lng_o,lat_o,lng_d,lat_d,line_distance
0,3000821,NaN,2018-11-02 17:54:30,2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-02 17:54:32,9,116.29,39.97,116.32,39.96,2793
1,2944522,NaN,2018-10-06 10:33:58,2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-06 10:34:23,9,116.31,39.93,116.27,40.00,8490
2,1677052,NaN,2018-10-23 01:02:45,2018-10-23 01:02:45,"[{""distance"": 21605, ""price"": 500, ""eta"": 5549...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-23 01:02:55,1,116.43,39.95,116.37,39.86,11233
3,986743,NaN,2018-11-12 13:52:31,2018-11-12 13:52:31,"[{""distance"": 30603, ""price"": 600, ""eta"": 4751...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-12 13:52:51,9,116.32,39.98,116.58,39.91,23540
4,184189,NaN,2018-10-20 12:52:07,2018-10-20 12:52:07,"[{""distance"": 4411, ""price"": 400, ""eta"": 2373,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-20 12:52:10,1,116.31,40.03,116.33,40.01,2801
5,3292454,NaN,2018-10-16 09:06:07,2018-10-16 09:06:07,"[{""distance"": 3437, ""price"": 1400, ""eta"": 858,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-16 09:06:40,1,116.30,39.94,116.27,39.96,3391
6,627704,NaN,2018-10-28 09:46:40,2018-10-28 09:46:40,"[{""distance"": 26609, ""price"": 600, ""eta"": 4057...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-10-28 09:46:53,2,116.29,39.86,116.45,39.93,15737
7,1548874,NaN,2018-11-28 08:34:30,2018-11-28 08:34:30,"[{""distance"": 5556, ""price"": 300, ""eta"": 2185,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-28 08:34:37,2,116.43,39.90,116.43,39.94,4441
8,1300125,NaN,2018-11-06 16:48:51,2018-11-06 16:48:51,"[{""distance"": 4366, ""price"": 200, ""eta"": 2056,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-06 16:48:57,1,116.51,39.92,116.55,39.91,3595
9,1806930,NaN,2018-11-29 11:08:52,2018-11-29 11:08:52,"[{""distance"": 22129, ""price"": 500, ""eta"": 4599...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018-11-29 11:09:04,2,116.31,40.03,116.46,39.96,14983
